In [973]:
import pandas as pd
import numpy as np 
import random
from util import get_split_cols, get_split_frame
from sklearn.preprocessing import StandardScaler

In [936]:
train = pd.read_csv("/data/project2/train.csv")

test = pd.read_csv("/data/project2/test.csv")
data = pd.read_csv("data.csv")
data = data.reset_index()
#train[train["Index"] == 0]
#train["index"] = train["Index"]
#train = train.drop(["Index"], axis=1)
#train = train.set_index("index")
#test[["Index"]].join(data, lsuffix='Index', rsuffix='index')[["index", "Index"]]
#pd.merge([data, train], right="asdf")
#data[test["Index"]]
true_y = pd.merge(data, test[["Index"]], left_on="index", right_on="Index")[["index", "Total Household Income"]]

In [937]:
num_train_cols, cat_train_cols = get_split_cols(train)
num_test_cols, cat_test_cols = get_split_cols(test)

In [938]:
assert(len(num_train_cols)==len(num_test_cols))
assert(len(cat_train_cols)==len(cat_test_cols))


In [945]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)

In [947]:
num_train_df, cat_train_df = get_split_frame(train)
num_test_df, cat_test_df = get_split_frame(test)
y = num_train_df['Total Household Income']
num_train_df = num_train_df.drop(["Total Household Income", "Index"], axis=1)
test_index = num_test_df[["Index"]]
num_test_df = num_test_df.drop(["Total Household Income", "Index"], axis=1)

In [1092]:
class Layer(object):
    def __init__(self, num_nodes, num_node_in_prev_layer, eta, activation='relu'):
        self.output = np.ones(num_nodes)
        self.weights = np.random.rand(num_nodes, num_node_in_prev_layer)
        self.deltas = np.ones(num_nodes)
        self.activation_type = activation
        self.eta = eta
        
    def forward(self, inputs, no_act=False):
        self.inputs = inputs
        self.sum_w_h = np.dot(self.weights, self.inputs)
        self.output = self.activation(self.sum_w_h)
        
    def activation(self, sum_w_h):
        if self.activation_type == "sigmoid":
            return 1 / (1 + np.exp(-sum_w_h))
        if self.activation_type == "relu":
            return np.maximum(sum_w_h, 0)
        return sum_w_h
    
    def activation_prime(self, fx):
        if self.activation_type == "sigmoid": 
            return fx*(1 - fx)
        if self.activation_type == "relu":
            return np.sign(fx)
        return -100
        
    def backward(self, prev_deltas):
        if self.activation_type != None:
            a_prime = self.activation_prime(self.output)
            self.delta = np.dot(prev_deltas * a_prime, self.inputs.T)
            self.next_delta = np.dot(self.weights.T, prev_deltas * a_prime)
        else:
            self.delta = np.dot(prev_deltas, self.inputs.T)
            self.next_delta = np.dot(self.weights.T, prev_deltas)
        self.update()


            
    def update(self):
        self.weights = self.weights - self.eta*self.delta


class NeuralNetwork(object):
    
    def __init__(self, num_hidden_layers, num_nodes_per_layer, num_inputs, num_outputs, eta=0.00001, act='relu'):
        self.network = [Layer(num_nodes=num_nodes_per_layer, num_node_in_prev_layer=num_inputs, eta=eta, activation=act)]
        self.network += [Layer(num_nodes=num_nodes_per_layer, num_node_in_prev_layer=num_nodes_per_layer, eta=eta, activation=act) for _ in range(num_hidden_layers - 1)]
        self.network += [Layer(num_nodes=num_outputs, num_node_in_prev_layer=num_nodes_per_layer,eta=eta, activation=None)]
        self.eta = eta 
        
    def forward(self, x):
        for layer in self.network:
            layer.forward(x)
            x = layer.output
            
    def backward(self, expected):
        deltas = self.network[-1].output - expected
        for layer in reversed(self.network):
            layer.backward(deltas)
            deltas = layer.next_delta
            
            
    def train(self, x, y, epoch, num_test_df, test_index, y_test, true_y, rows):
        prev_err = np.inf
        for i in range(epoch):
            print("\nEpoch: ", i)
            self.sum_error = 0 
            for xi, yi in zip(x,y):
                self.forward(np.array([xi]).T)
                self.sum_error += (self.network[-1].output - yi)**2
                if self.sum_error > prev_err or (abs(prev_err - self.sum_error) < 1):
                    print("Out Sum Error: ", self.sum_error)  
                    print("Epochs: ", i)
                    print(self.rmse(num_test_df, test_index, y_test, true_y, rows))
                    return
                self.backward(np.array([yi]))
            prev_err = self.sum_error
            print("Sum Error: ", self.sum_error)   
            self.rmse(num_test_df, test_index, y_test, true_y, rows)


    def rmse(self, num_test_df, test_index, y_test, true_y, rows):
        y_test = []
        #rows = random.sample(df.index, 10)
        x = scalar_x.fit_transform(num_test_df.values)
        for x_row in x:
            y_test.append(scalar_y.inverse_transform(self.predict(x_row))[0])

        test_out = pd.DataFrame()
        test_out['index'] = test_index['Index']
        test_out['Total Household Income'] = y_test
        test_out.sort_values("index", inplace=True)
        test_out.set_index("index", inplace=True)
        rmse = np.sqrt(sum((true_y.ix[rows]["Total Household Income"] - test_out.ix[rows]["Total Household Income"])**2)/int(.3*len(true_y)))
        print("rsme: ", rmse)


            
    def predict(self, x):
        self.forward(x)
        return self.network[-1].output

In [1096]:
scalar_x = StandardScaler()
scalar_y = StandardScaler()
x = scalar_x.fit_transform(num_train_df.values)
yy = scalar_y.fit_transform(y.values.reshape(-1,1))
rows = random.sample(set(true_y.index), int(.3*len(true_y)))
final = NeuralNetwork(1, 100, 44, 1, .0001, "relu")
final.train(x, yy, 200, num_test_df, test_index, y_test, true_y, rows)
final.predict(np.array([x[0]]).T)

/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)



Epoch:  0
Sum Error:  [[ 16040909.38313341]]
rsme:  333072.195872

Epoch:  1
Sum Error:  [[ 32449.09543993]]
rsme:  262724.270639

Epoch:  2
Sum Error:  [[ 22194.22502166]]
rsme:  229200.329921

Epoch:  3
Sum Error:  [[ 17242.20202962]]
rsme:  210570.147769

Epoch:  4
Sum Error:  [[ 14519.36688678]]
rsme:  198527.730495

Epoch:  5
Sum Error:  [[ 12856.03456109]]
rsme:  189829.51646

Epoch:  6
Sum Error:  [[ 11675.55293863]]
rsme:  183504.930237

Epoch:  7
Sum Error:  [[ 10762.74659198]]
rsme:  173264.015451

Epoch:  8
Sum Error:  [[ 9787.0673343]]
rsme:  150256.454496

Epoch:  9
Sum Error:  [[ 9160.26844132]]
rsme:  142780.778561

Epoch:  10
Sum Error:  [[ 8824.83677397]]
rsme:  138707.169066

Epoch:  11
Sum Error:  [[ 8567.80481536]]
rsme:  135695.638391

Epoch:  12
Sum Error:  [[ 8359.28894472]]
rsme:  133259.097656

Epoch:  13
Sum Error:  [[ 8185.79456424]]
rsme:  131213.766647

Epoch:  14
Sum Error:  [[ 8037.70593022]]
rsme:  129457.272877

Epoch:  15
Sum Error:  [[ 7910.07824428]

array([[-0.23880383]])

In [1098]:
y_test = []
#rows = random.sample(df.index, 10)
x = scalar_x.fit_transform(num_test_df.values)
for x_row in x:
    y_test.append(scalar_y.inverse_transform(final.predict(x_row))[0])

test_out = pd.DataFrame()
test_out['index'] = test_index['Index']
test_out['Total Household Income'] = y_test
test_out.sort_values("index", inplace=True)
test_out.set_index("index", inplace=True)

/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [1107]:
test_out.reset_index().to_csv("1L100NRelu1863MSE.csv", index=False)

In [1104]:
final.rmse(num_test_df, test_index, y_test, true_y, rows)

rsme:  100469.262591


/opt/conda/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [1014]:
true_y.ix[rows]

,Total Household Income
index,
40109,172370
20524,340118
3765,89810
7073,100900
18201,108651
14841,368752
23621,69128
37759,790172
33858,115992


In [ ]:
data = np.array([[1, 1],[2, 2],[3, 3],[4, 4],[5, 5],[6, 6],[7, 7],[8, 8],[9, 9],[10, 10],[11, 11],[12, 12],[13, 13],[14, 14],[15, 15],[16, 16],[17, 17],[18, 18],[19, 19],[20, 20],[21, 21],[22, 22],[23, 23],[24, 24],[25, 25],[26, 26],[27, 27],[28, 28],[29, 29],[30, 30],[31, 31],[32, 32],[33, 33],[34, 34],[35, 35],[36, 36],[37, 37],[38, 38],[39, 39],[40, 40],[41, 41],[42, 42],[43, 43],[44, 44],[45, 45],[46, 46],[47, 47],[48, 48],[49, 49],[50, 50],[51, 51],[52, 52],[53, 53],[54, 54],[55, 55],[56, 56],[57, 57],[58, 58],[59, 59],[60, 60],[61, 61],[62, 62],[63, 63],[64, 64],[65, 65],[66, 66],[67, 67],[68, 68],[69, 69],[70, 70],[71, 71],[72, 72],[73, 73],[74, 74],[75, 75],[76, 76],[77, 77],[78, 78],[79, 79],[80, 80],[81, 81],[82, 82],[83, 83],[84, 84],[85, 85],[86, 86],[87, 87],[88, 88],[89, 89],[90, 90],[91, 91],[92, 92],[93, 93],[94, 94],[95, 95],[96, 96],[97, 97],[98, 98],[99, 99],[100, 100]])
expected = np.array([2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48,50,52,54,56,58,60,62,64,66,68,70,72,74,76,78,80,82,84,86,88,90,92,94,96,98,100,102,104,106,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168,170,172,174,176,178,180,182,184,186,188,190,192,194,196,198,200])
nn = NeuralNetwork(1, 3, 2, 1, .00000001, "relu")
nn.predict(np.array([[3,6]]).T)